In [ ]:
import numpy as np
import matplotlib
import csv
from matplotlib import pyplot
%run lstm.ipynb

In [ ]:
# Функция вычитывает заданный столбец CSV файла.
def read_csv_column(filename, col_id):
    file = open(filename)
    reader = csv.reader(file)
    data = [float(row[col_id]) for row in reader]
    file.close()
    return data

In [ ]:
# Функция реализует разбиение данных на два набора в указанных пропорциях.
def split_data(data, ratio=0.8):
    num_samples = int(len(data) * ratio)
    return data[:num_samples], data[num_samples:]

In [ ]:
# Открываем CSV файл и считываем третью колонку - там лежат нужные нам данные.
filename = 'e:\international-airline-passengers-monthly-totals-inthousands-jan-49-dec-60.csv'
data = read_csv_column(filename, 2)
train_data, test_data = split_data(data, ratio=0.8)

# Нормализуем данные
data_max = np.amax(data)
train_data /= data_max
test_data /= data_max

# Рисуем на одном грпфике часть обучения и часть для тестов.
matplotlib.pyplot.plot(np.arange(0, len(train_data), 1), train_data)
matplotlib.pyplot.plot(np.arange(0, len(test_data), 1) + len(train_data), test_data)

In [ ]:
# Гиперпараметры.
seq_size = 5
learn_rate = 0.5

train_x, train_y = [], []
for i in range(len(train_data) - seq_size):
    train_x.append(train_data[i:i+seq_size])
    train_y.append(train_data[i+1:i+1+seq_size])

test_x, test_y = [], []
for i in range(len(test_data) - seq_size):
    test_x.append(test_data[i:i+seq_size])
    test_y.append(test_data[i+1:i+1+seq_size])

In [ ]:
sess = tf.Session()
net = lstm_cell(sess, 5, learn_rate)

for e in range(20):
    net.clear_state()
    for x, y in zip(train_x, train_y):
        loss = net.train([x], [y], repeat=10)

In [ ]:
prev_seq = [train_x[-1]]
#prev_seq = [test_x[0]]
pred_seq = []

for i in range(30):
    next_seq = net.test(prev_seq)
    prev_seq = next_seq
    pred_seq.append(next_seq)

In [ ]:
pred_curve = []
for i in pred_seq:
    pred_curve.append(i[0,-1])

In [ ]:
matplotlib.pyplot.plot(np.arange(0, len(train_data), 1), train_data)
matplotlib.pyplot.plot(np.arange(0, len(test_data), 1) + len(train_data), test_data)
matplotlib.pyplot.plot(np.arange(len(train_data), len(train_data) + len(pred_curve), 1), pred_curve)